In [1]:
import plotly.express as px
import pandas as pd
import json

In [4]:
geo_data=""
with open("../data/raw/countries.geojson", "r") as f:
    geojson=json.load(f)
# geojson = json.load("./countries.geojson")
ds = pd.read_csv('../data/raw/share-of-individuals-using-the-internet.csv')

In [5]:
ds.sample(12)

,Entity,Code,Year,Individuals using the Internet (% of population)
6048,South Korea,KOR,1993,0.249947
2913,IDA & IBRD total,NaN,2011,23.404839
3718,Liberia,LBR,1997,0.004489
1173,Chad,TCD,2010,1.700000
2953,IDA only,NaN,1997,0.014040
1490,Czechia,CZE,2006,47.930000
5587,Samoa,WSM,1995,0.000000
6290,Sweden,SWE,2005,84.830000
2788,Honduras,HND,2002,2.597403
6589,Turkey,TUR,1999,2.292118


In [9]:
geojson.keys()

dict_keys(['type', 'features'])

we Rename the column form Individuals using the Internet (% of population) to Internet_usage

In [10]:
ds.rename(columns={'Individuals using the Internet (% of population)':"Internat_usage"} ,inplace=True)

we need to make sure that all countries have the same Name in the geodata as will in the dataset, otherwise we will get a mismatch between the geodata and the dataset. We can do this by renaming

In [11]:
country_code_name={}
for i in range(len(geojson["features"])):
  # if geojson["features"][i]["properties"]["ISO_A3"] not in df3['Code'].values:
  country_code_name[geojson["features"][i]["properties"]['ISO_A3']]=geojson["features"][i]["properties"]['ADMIN'] 
      
# len(country)   
country_code_name 

{'ABW': 'Aruba',
 'AFG': 'Afghanistan',
 'AGO': 'Angola',
 'AIA': 'Anguilla',
 'ALB': 'Albania',
 'ALA': 'Aland',
 'AND': 'Andorra',
 'ARE': 'United Arab Emirates',
 'ARG': 'Argentina',
 'ARM': 'Armenia',
 'ASM': 'American Samoa',
 'ATA': 'Antarctica',
 '-99': 'Akrotiri Sovereign Base Area',
 'ATF': 'French Southern and Antarctic Lands',
 'ATG': 'Antigua and Barbuda',
 'AUS': 'Australia',
 'AUT': 'Austria',
 'AZE': 'Azerbaijan',
 'BDI': 'Burundi',
 'BEL': 'Belgium',
 'BEN': 'Benin',
 'BFA': 'Burkina Faso',
 'BGD': 'Bangladesh',
 'BGR': 'Bulgaria',
 'BHR': 'Bahrain',
 'BHS': 'The Bahamas',
 'BIH': 'Bosnia and Herzegovina',
 'BLM': 'Saint Barthelemy',
 'BLR': 'Belarus',
 'BLZ': 'Belize',
 'BMU': 'Bermuda',
 'BOL': 'Bolivia',
 'BRA': 'Brazil',
 'BRB': 'Barbados',
 'BRN': 'Brunei',
 'BTN': 'Bhutan',
 'BWA': 'Botswana',
 'CAF': 'Central African Republic',
 'CAN': 'Canada',
 'CHE': 'Switzerland',
 'CHL': 'Chile',
 'CHN': 'China',
 'CIV': 'Ivory Coast',
 'CMR': 'Cameroon',
 'COD': 'Democratic

I will make a cope of the data before I apply the rename function. This will allow me to keep the original data intact and make sure that I can revert back

In [12]:
ds_copy=ds
ds_copy["Country"]=ds_copy['Code'].map(country_code_name)
ds_copy.sample(3)

,Entity,Code,Year,Internat_usage,Country
6519,Tonga,TON,2015,38.651272,Tonga
5452,Russia,RUS,2000,1.977230,Russia
568,Belgium,BEL,2000,29.431692,Belgium


Now I need to campier the country the are in the dataset und the counties in the geodata

In [20]:
not_matsh={}
for index, row in ds_copy.iterrows(): # Use iterrows() correctly
    if row['Code'] not in country_code_name.keys():
        not_matsh[row['Code']] = row['Entity']
not_matsh      

{nan: 'Upper middle income', 'OWID_KOS': 'Kosovo', 'OWID_WRL': 'World'}

In [22]:
not_matsh2={}
for index, value in enumerate(country_code_name): # Use iterrows() correctly
    if row['Code'] not in ds_copy["Code"].unique():
        not_matsh2[index] = value
not_matsh2

{}

the updated Country in the dataset to identify the countries

In [25]:
country_set = set(ds_copy['Country'].unique())
entity_set = set(ds_copy['Entity'].unique())

# Find elements in 'Country' but not in 'Entity'
difference = entity_set - country_set

# Print or further process the difference
difference

{'Africa Eastern and Southern',
 'Africa Western and Central',
 'Arab World',
 'Bahamas',
 'Caribbean Small States',
 'Central Europe and the Baltics',
 'Congo',
 "Cote d'Ivoire",
 'Curacao',
 'Czechia',
 'Democratic Republic of Congo',
 'Early-demographic dividend',
 'East Asia & Pacific',
 'East Asia & Pacific (IDA & IBRD)',
 'East Asia & Pacific (excluding high income)',
 'Eswatini',
 'Euro area',
 'Europe & Central Asia',
 'Europe & Central Asia (IDA & IBRD)',
 'Europe & Central Asia (excluding high income)',
 'European Union',
 'Faeroe Islands',
 'Fragile and conflict affected situations',
 'Guinea-Bissau',
 'Heavily indebted poor countries (HIPC)',
 'High income',
 'Hong Kong',
 'IBRD only',
 'IDA & IBRD total',
 'IDA blend',
 'IDA only',
 'IDA total',
 'Kosovo',
 'Late-demographic dividend',
 'Latin America & Caribbean',
 'Latin America & Caribbean (IDA & IBRD)',
 'Latin America & Caribbean (excluding high income)',
 'Least developed countries: UN classification',
 'Low & middle

In [26]:
len(difference)

66

here are the country the are in geodata but there are't in the dataset

In [27]:
country_set = set(ds_copy['Country'].unique())
entity_set = set(country_code_name.values())

# Find elements in 'Country' but not in 'Entity'
difference = entity_set - country_set

# Print or further process the difference
difference

{'Akrotiri Sovereign Base Area',
 'Aland',
 'Anguilla',
 'Antarctica',
 'British Indian Ocean Territory',
 'Cook Islands',
 'Falkland Islands',
 'French Southern and Antarctic Lands',
 'Guernsey',
 'Heard Island and McDonald Islands',
 'Isle of Man',
 'Jersey',
 'Montserrat',
 'Niue',
 'Norfolk Island',
 'Pitcairn Islands',
 'Saint Barthelemy',
 'Saint Helena',
 'Saint Martin',
 'Saint Pierre and Miquelon',
 'Sint Maarten',
 'South Georgia and South Sandwich Islands',
 'Taiwan',
 'United States Minor Outlying Islands',
 'Vatican',
 'Wallis and Futuna',
 'Western Sahara'}

In [ ]:
ds_copy[ds_copy['Country']==]

,Entity,Code,Year,Internat_usage,Country


In [49]:
ds_copy['Country'].value_counts().sort_values(ascending=True)

KeyError: 'Country'

In [39]:
ds_copy[ds_copy['Country']==" "]

,Entity,Code,Year,Internat_usage,Country


In [47]:
# ds_copy.drop(columns=['Entity'],inplace=True).rename({'Country':"Entity"})
ds_copy.rename(columns={'Internat_usage':"Internet_usage","Country":"Entity"},inplace=True)

In [48]:
ds_copy.head()

,Code,Year,Internet_usage,Entity
0,AFG,1990,0.0,Afghanistan
1,AFG,1991,0.0,Afghanistan
2,AFG,1992,0.0,Afghanistan
3,AFG,1993,0.0,Afghanistan
4,AFG,1994,0.0,Afghanistan


I need to reorganized  the datafram structure 

In [50]:
ds_copy=ds_copy[["Entity","Code","Year","Internet_usage"]]

In [51]:
ds_copy.head()

,Entity,Code,Year,Internet_usage
0,Afghanistan,AFG,1990,0.0
1,Afghanistan,AFG,1991,0.0
2,Afghanistan,AFG,1992,0.0
3,Afghanistan,AFG,1993,0.0
4,Afghanistan,AFG,1994,0.0


In [52]:
!pip install pycountry
!pip install pycountry_convert
import pycountry
import pycountry_convert as pc

# Create a function to get the continent from a country code
def get_continent(country_code):
    try:
        country_alpha2 = pycountry.countries.get(alpha_3=country_code).alpha_2
        # Get the continent code using pycountry_convert
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        # Get the continent name from the continent code
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except (KeyError, AttributeError):
        return 'Unknown'

get_continent("USA")

'North America'

In [55]:
ds_copy.head()

,Entity,Code,Year,Internet_usage
0,Afghanistan,AFG,1990,0.0
1,Afghanistan,AFG,1991,0.0
2,Afghanistan,AFG,1992,0.0
3,Afghanistan,AFG,1993,0.0
4,Afghanistan,AFG,1994,0.0


In [58]:
def get_continent(country_code):
    try:
        country_alpha2 = pycountry.countries.get(alpha_3=country_code).alpha_2
        # Get the continent code using pycountry_convert
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        # Get the continent name from the continent code
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except (KeyError, AttributeError):
        return 'Unknown'

get_continent("USA")

'North America'

In [ ]:
ds_copy['Continent'] = ds_copy['Code'].apply(get_continent)


In [60]:
df_int=pd.read_csv("../data/raw/internet_usages_continent.csv")
df_int.head()

,Entity,Code,Year,Internet_usage,continent
0,Afghanistan,AFG,1990,0.0,Asia
1,Afghanistan,AFG,1991,0.0,Asia
2,Afghanistan,AFG,1992,0.0,Asia
3,Afghanistan,AFG,1993,0.0,Asia
4,Afghanistan,AFG,1994,0.0,Asia
